<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timegan_exchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install synthcity tsbootstrap

# Training

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader
from synthcity.utils.serialization import save_to_file, load_from_file

log.add(sink=sys.stderr, level="INFO")

Mounted at /content/drive
[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


In [3]:
file_path = "/content/drive/Shareddrives/sp_env/datasets/Exchange Rate/exchange_rate.txt"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print('missing values:', df.isnull().sum())

   0.785500  1.611000  0.861698  0.634196  0.211242  0.006838  0.593000  \
0    0.7818    1.6100  0.861104  0.633513  0.211242  0.006863    0.5940   
1    0.7867    1.6293  0.861030  0.648508  0.211242  0.006975    0.5973   
2    0.7860    1.6370  0.862069  0.650618  0.211242  0.006953    0.5970   
3    0.7849    1.6530  0.861995  0.656254  0.211242  0.006940    0.5985   
4    0.7866    1.6537  0.861030  0.654879  0.211242  0.006887    0.6040   

   0.525486  
0  0.523972  
1  0.526316  
2  0.523834  
3  0.527426  
4  0.526177  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7587 entries, 0 to 7586
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0.785500  7587 non-null   float64
 1   1.611000  7587 non-null   float64
 2   0.861698  7587 non-null   float64
 3   0.634196  7587 non-null   float64
 4   0.211242  7587 non-null   float64
 5   0.006838  7587 non-null   float64
 6   0.593000  7587 non-null   float64
 7   0.5

In [4]:
# for exchange dataset only
df.columns = [
    'Australia', 'British_Pound', 'Canada', 'Switzerland',
    'China', 'Japan', 'New_Zealand', 'Singapore'
]

# set seed and randomly choose 2 features
np.random.seed(42)

# Filter the dataframe to only include selected features
selected_features = np.random.choice(df.columns, size=2, replace=False)
print(f"Selected features: {selected_features}")

# Immediately filter your full DataFrame down to just those two series:
df = df[selected_features]

Selected features: ['British_Pound' 'Japan']


In [5]:
# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # use later for LSTM-TSTR

# Normalize the data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_test, columns=df_test.columns, index=df_test.index)


# set sequence length to 30 to capture monthly patterns
# dataset = daily
sequence_length = 30
# temporal_data = []
# observation_times = []

# for start in range(len(df_scaled_train) - sequence_length + 1):
#     sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
#     temporal_data.append(sequence)
#     observation_times.append(list(range(sequence_length)))  # relative time within the window


# # Dummy outcome for data loader
# dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# # --- Create DataLoader for TimeGAN ---
# loader = TimeSeriesDataLoader(
#     temporal_data=temporal_data,  # List of sequences (DataFrames)
#     observation_times=observation_times,  # List of time indices (DataFrames)
#     static_data=None,  # No static data for now (can be set if needed)
#     outcome=dummy_outcome,  # Dummy outcome for forecasting
# )

# # Print the loader info
# print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")

In [6]:
temporal_data_test = []
observation_times_test = []

# Generate sequences from df_scaled_test only
for start in range(len(df_scaled_test) - sequence_length + 1):
    sequence = df_scaled_test.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data_test.append(sequence)
    observation_times_test.append(list(range(sequence_length)))  # relative time within the window

# Dummy outcome for TimeGAN (can be used in DataLoader)
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data_test)), columns=["outcome"])

# Create DataLoader for TimeGAN
loader_test = TimeSeriesDataLoader(
    temporal_data=temporal_data_test,
    observation_times=observation_times_test,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader TEST SET created with {len(temporal_data_test)} sequences")

TimeSeriesDataLoader TEST SET created with 1471 sequences


In [ ]:
print(len(df_train))  # Check the length of the dataframe
print(loader.dataframe())

In [ ]:
from synthcity.plugins import Plugins

hparams = {
          "mode": "LSTM",
}

# Load TimeGAN with custom parameters
syn_model = Plugins().get("timegan", **hparams)

[2025-04-26T13:20:23.621941+0000][2010][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-04-26T13:20:23.621941+0000][2010][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py


In [ ]:
# Print all parameters of initialized model
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeGANPlugin
clipping_value: 0
compress_dataset: False
dataloader_sampling_strategy: imbalanced_time_censoring
device: cuda
discriminator_batch_norm: False
discriminator_dropout: 0.1
discriminator_loss: None
discriminator_lr: 0.001
discriminator_n_iter: 1
discriminator_n_layers_hidden: 3
discriminator_n_units_hidden: 300
discriminator_nonlin: leaky_relu
discriminator_weight_decay: 0.001
embedding_penalty: 10
encoder: None
encoder_max_clusters: 20
expecting_conditional: False
fitted: False
gamma_penalty: 1
generator_batch_norm: False
generator_dropout: 0.01
generator_loss: None
generator_lr: 0.001
generator_n_layers_hidden: 2
generator_n_units_hidden: 150
generator_nonlin: leaky_relu
generator_nonlin_out_continuous: tanh
generator_nonlin_out_discrete: softmax
generator_residual: True
generator_weight_decay: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timegan
module_relative_path: ../time_series/plugin_timegan.py
moments_penalty: 100
n_i

## fitting the model

In [ ]:
print(loader.shape)

# --- Train the model ---
syn_model.fit(loader)

(104130, 5)


100%|██████████| 1000/1000 [1:56:52<00:00,  7.01s/it]


In [ ]:
#@title optional: generate data from saved model
from synthcity.utils.serialization import save_to_file, load_from_file

# Save model to drive
save_to_file('/content/drive/Shareddrives/sp_env/saved_models/GAN_Exchange.pkl', syn_model)

# Load the model
# syn_model = load_from_file('/content/drive/Shareddrives/sp_env/synmodel.pkl')

In [ ]:
# --- Generate Synthetic Data ---
n_samples = len(temporal_data)
syn_data = syn_model.generate(count=n_samples)
print(syn_data.shape)

(100017, 5)


In [ ]:
# --- Save with automated format ---
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-3500.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

Synthetic data saved to: /content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv


# Evaluation

## Prerequisites

In [17]:
syn_data = pd.read_csv('/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv')

In [18]:
# Define selected columns explicitly
# selected_columns = ['seq_temporal_HUFL', 'seq_temporal_HULL', 'seq_temporal_LUFL', 'seq_temporal_LULL', 'seq_temporal_MUFL', 'seq_temporal_MULL', 'seq_temporal_OT']
selected_columns = ['seq_temporal_British_Pound','seq_temporal_Japan']
# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader_test.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data[selected_columns].to_numpy()

In [28]:
print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

       seq_temporal_British_Pound  seq_temporal_Japan
0                        0.307506            0.871024
1                        0.305945            0.881916
2                        0.305945            0.885201
3                        0.297145            0.892808
4                        0.295086            0.892289
...                           ...                 ...
44125                   -0.203774            0.187068
44126                   -0.202059            0.203492
44127                   -0.187599            0.197787
44128                   -0.187879            0.196577
44129                   -0.187879            0.196577

[44130 rows x 2 columns] 
 ------------------------------------------------------- 
 [[0.25763111 0.18547966]
 [0.0866552  0.18546773]
 [0.08670829 0.18559386]
 ...
 [0.31743031 0.1863488 ]
 [0.25738601 0.18644113]
 [0.61835132 0.18653295]]
<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
(44130, 2) (100017, 2)


## Generate distance metrics

### Helper Functions

In [24]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [ ]:
# Assuming df_scaled is the DataFrame containing your scaled ETD data

# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(44130, 2) (44130, 2)
0.08117466787512753
0.15334526304995016
Wasserstein Distance Results:
{'seq_temporal_British_Pound': 0.08117466787512753, 'seq_temporal_Japan': 0.15334526304995016}
17.43605457568381
17.20093945765117
KL Divergence Results:
{'seq_temporal_British_Pound': 17.43605457568381, 'seq_temporal_Japan': 17.20093945765117}


# LSTM downstream

In [20]:
real_data = loader_test.dataframe()
df_synth = pd.read_csv("/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/exchange/042625-152949-timeganplugin-3500.csv")

# drop unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

print(f"real_data: {real_data.shape}, synthetic_data: {df_synth.shape}")

real_data: (44130, 2), synthetic_data: (100017, 2)


In [ ]:
real_data = loader_test.dataframe()
df_synth = syn_data

# 2. Drop the unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [10]:
# Convert to tensors (float32 for PyTorch)
data_real = torch.tensor(real_data.values, dtype=torch.float32)
data_synth = torch.tensor(df_synth.values, dtype=torch.float32)

#  Sequence builder
def make_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return torch.stack(X), torch.stack(y)

SEQ_LEN = sequence_length

# Sequences for synthetic (train)
X_train, y_train = make_sequences(data_synth, SEQ_LEN)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

# Sequences for real (test)
X_test, y_test = make_sequences(data_real, SEQ_LEN)

In [11]:

# ─── Model Definition ──────────────────────────────────────
class ShallowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn shape: (1, batch, hidden_size)
        out = self.linear(hn.squeeze(0))  # squeeze to (batch, hidden_size)
        return out


# ─── Model Init ─────────────────────────────────────────────
model = ShallowLSTM(input_size=X_train.shape[2], hidden_size=64)

# ─── Optimizer & Loss ───────────────────────────────────────
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [12]:
# ─── Training ───────────────────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0 or epoch == 1:
    print(f"Epoch {epoch}: Train MSE = {loss.item():.6f}")

Epoch 1: Train MSE = 0.005228
Epoch 2: Train MSE = 0.015128
Epoch 3: Train MSE = 0.007854
Epoch 4: Train MSE = 0.003106
Epoch 5: Train MSE = 0.005161
Epoch 6: Train MSE = 0.003355
Epoch 7: Train MSE = 0.004467
Epoch 8: Train MSE = 0.011657
Epoch 9: Train MSE = 0.009701
Epoch 10: Train MSE = 0.007955
Epoch 11: Train MSE = 0.007146
Epoch 12: Train MSE = 0.016024
Epoch 13: Train MSE = 0.025980
Epoch 14: Train MSE = 0.008415
Epoch 15: Train MSE = 0.009838
Epoch 16: Train MSE = 0.005079
Epoch 17: Train MSE = 0.005687
Epoch 18: Train MSE = 0.002633
Epoch 19: Train MSE = 0.008589
Epoch 20: Train MSE = 0.006242
Epoch 21: Train MSE = 0.008448
Epoch 22: Train MSE = 0.009656
Epoch 23: Train MSE = 0.001432
Epoch 24: Train MSE = 0.007562
Epoch 25: Train MSE = 0.002765
Epoch 26: Train MSE = 0.002328
Epoch 27: Train MSE = 0.011676
Epoch 28: Train MSE = 0.007127
Epoch 29: Train MSE = 0.013822
Epoch 30: Train MSE = 0.010113
Epoch 31: Train MSE = 0.006127
Epoch 32: Train MSE = 0.008170
Epoch 33: Train M

In [13]:
# --- ADD THESE LINES TO SAVE THE MODEL ---
# Define a path where you want to save your model.
# Use a meaningful name, especially for TRTR vs. TSTR models.
# Example for TRTR Electricity model:
MODEL_SAVE_PATH = '/content/drive/Shareddrives/sp_env/saved_models/LSTM/tstr_GAN_exchange.pth'

# Save only the model's learned parameters (state_dict)
torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"Model saved to: {MODEL_SAVE_PATH}")

Model saved to: /content/drive/Shareddrives/sp_env/saved_models/LSTM/tstr_GAN_exchange.pth


In [14]:
#@title ✧.* model evaluation ✧.*
model.eval()
with torch.no_grad():
    preds = model(X_test)
    test_mse = loss_fn(preds, y_test).item()
    test_mae = mean_absolute_error(y_test.numpy(), preds.numpy())

    print(f"Test MSE: {test_mse:.6f}")
    print(f"Test MAE: {test_mae:.6f}")

Test MSE: 0.034710
Test MAE: 0.143574


# Bootstrapping

In [21]:
from tsbootstrap import MovingBlockBootstrap
import numpy as np

bootstrap_configs = {
    "weather": {"block_length": 36, "n_bootstraps": 15, "rng": 42},       # 6-hour pattern (10-min interval)
    "electricity": {"block_length": 24, "n_bootstraps": 15, "rng": 42},   # 1-day pattern (hourly)
    "exchange": {"block_length": 30, "n_bootstraps": 15, "rng": 42},      # 1-month pattern (daily)
}

# Example for weather
dataset_name = "exchange"
config = bootstrap_configs[dataset_name]

real_test_array = real_data[selected_columns].to_numpy()  # shape (N, features)
mbb = MovingBlockBootstrap(
    n_bootstraps=config["n_bootstraps"],
    rng=config["rng"],
    block_length=config["block_length"]
)
boot_samples = mbb.bootstrap(real_test_array, return_indices=False)

In [22]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

<All keys matched successfully>

In [25]:
bootstrap_results = []

for b_idx, boot_real in enumerate(boot_samples):
    # 1. Match the synthetic data size
    syn_trimmed = synthetic_data[:len(boot_real)]

    # 2. Fidelity metrics
    wasserstein = compute_wasserstein(boot_real, syn_trimmed, selected_columns)
    kl = compute_kl_divergence(boot_real, syn_trimmed, selected_columns)

    # 3. Utility metrics
    # Preprocess this bootstrap sample for LSTM (as you do with real_data)
    boot_tensor = torch.tensor(boot_real, dtype=torch.float32)
    Xb_test, yb_test = make_sequences(boot_tensor, SEQ_LEN)

    model.eval()
    with torch.no_grad():
        preds = model(Xb_test)
        mse = mean_squared_error(yb_test.numpy(), preds.numpy())
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(yb_test.numpy(), preds.numpy())

    # 4. Store results
    bootstrap_results.append({
        'bootstrap': b_idx,
        'wasserstein': np.mean(list(wasserstein.values())),
        'kl': np.mean(list(kl.values())),
        'rmse': rmse,
        'mae': mae
    })

(44130, 2) (44130, 2)
0.08140693101833461
0.151777377023289
17.570698564722292
17.093488783171985
(44130, 2) (44130, 2)
0.07904870100812919
0.15336695594370459
17.404968717813574
17.111706387512072
(44130, 2) (44130, 2)
0.07371995443513063
0.1550851659946605
17.604149779229555
17.497905603480316
(44130, 2) (44130, 2)
0.0783790714929946
0.1528026503834652
17.55428492841301
17.18087286787068
(44130, 2) (44130, 2)
0.08656598937119925
0.15277463234966976
17.150676311522567
17.387680415791646
(44130, 2) (44130, 2)
0.07948165084286721
0.1551900434558216
17.464554902022062
17.39558241925526
(44130, 2) (44130, 2)
0.08018590411887444
0.14861175137008495
17.500306343573524
17.070677191868725
(44130, 2) (44130, 2)
0.07661886504049417
0.15364064790638426
17.540829620098332
17.46420603818756
(44130, 2) (44130, 2)
0.08574885338852964
0.14794079273658156
17.18785140096761
17.136194500628374
(44130, 2) (44130, 2)
0.08520386206932949
0.1577888993107748
17.072698051273328
16.9007159655047
(44130, 2) (44

In [31]:
print(bootstrap_results)

# Assuming bootstrap_results is your list of dicts
df_results = pd.DataFrame(bootstrap_results)

df_results['Dataset'] = 'exchange'
df_results['Model'] = 'TimeGAN'

df_results

[{'bootstrap': 0, 'wasserstein': 0.1165921540208118, 'kl': 17.33209367394714, 'rmse': 0.18541616, 'mae': 0.1432341}, {'bootstrap': 1, 'wasserstein': 0.11620782847591689, 'kl': 17.258337552662823, 'rmse': 0.18523572, 'mae': 0.14206706}, {'bootstrap': 2, 'wasserstein': 0.11440256021489556, 'kl': 17.551027691354935, 'rmse': 0.18230224, 'mae': 0.14124227}, {'bootstrap': 3, 'wasserstein': 0.1155908609382299, 'kl': 17.367578898141844, 'rmse': 0.18615931, 'mae': 0.14273407}, {'bootstrap': 4, 'wasserstein': 0.1196703108604345, 'kl': 17.269178363657105, 'rmse': 0.18995214, 'mae': 0.14630097}, {'bootstrap': 5, 'wasserstein': 0.1173358471493444, 'kl': 17.43006866063866, 'rmse': 0.18965985, 'mae': 0.14400981}, {'bootstrap': 6, 'wasserstein': 0.1143988277444797, 'kl': 17.285491767721126, 'rmse': 0.1824454, 'mae': 0.14063036}, {'bootstrap': 7, 'wasserstein': 0.11512975647343922, 'kl': 17.502517829142946, 'rmse': 0.18376955, 'mae': 0.14203142}, {'bootstrap': 8, 'wasserstein': 0.1168448230625556, 'kl'

,bootstrap,wasserstein,kl,rmse,mae,Dataset,Model
0,0,0.116592,17.332094,0.185416,0.143234,exchange,TimeGAN
1,1,0.116208,17.258338,0.185236,0.142067,exchange,TimeGAN
2,2,0.114403,17.551028,0.182302,0.141242,exchange,TimeGAN
3,3,0.115591,17.367579,0.186159,0.142734,exchange,TimeGAN
4,4,0.119670,17.269178,0.189952,0.146301,exchange,TimeGAN
5,5,0.117336,17.430069,0.189660,0.144010,exchange,TimeGAN
6,6,0.114399,17.285492,0.182445,0.140630,exchange,TimeGAN
7,7,0.115130,17.502518,0.183770,0.142031,exchange,TimeGAN
8,8,0.116845,17.162023,0.184547,0.143151,exchange,TimeGAN
9,9,0.121496,16.986707,0.191680,0.147266,exchange,TimeGAN


In [32]:
summary_row = {
    'Dataset': 'exchange',
    'Model': 'TimeGAN',
    'Wasserstein': df_results['wasserstein'].mean(),
    'KL': df_results['kl'].mean(),
    'RMSE': df_results['rmse'].mean(),
    'MAE': df_results['mae'].mean()
}

df_summary = pd.DataFrame([summary_row])
print(df_summary)

    Dataset    Model  Wasserstein         KL      RMSE       MAE
0  exchange  TimeGAN     0.117161  17.302635  0.186563  0.143466
